In [3]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [4]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [5]:
# default_exp memory.tensorboard

In [12]:
# export
# Python native modules
import os
from fnmatch import fnmatch
# Third party libs
from fastcore.all import *
from tensorboard.backend.event_processing import event_accumulator
# Local modules
from fastrl.core import *

# Memory Tensorboard
> Tensorboard / Plotly visualizations using tensorboard as a backend within the context of memory

In [7]:
# hide
SHOW_TENSOR_BOARD=True
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

Selecting TensorBoard with logdir runs (started 20:00:56 ago; port 6006, pid 868).


In [11]:
ea = event_accumulator.EventAccumulator('runs/Oct23_14-46-31_9b89ef136e90_dqn_target/events.out.tfevents.1635000391.9b89ef136e90.114.4',
  size_guidance={ # see below regarding this argument
      event_accumulator.COMPRESSED_HISTOGRAMS: 500,
      event_accumulator.IMAGES: 0,
      event_accumulator.AUDIO: 4,
      event_accumulator.SCALARS: 0,
      event_accumulator.HISTOGRAMS: 1,
  })

In [ ]:
# export
def get_event_accumulator(path:Path # Possible partial path to load events from.
                          pattern:Optional[str]=None, # pattern to load the files
                         ):
    if in_('*',path): pattern

In [8]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/10b_agents.dqn.targets.ipynb
converting: /home/fastrl_user/fastrl/nbs/00_core.ipynb
converting: /home/fastrl_user/fastrl/nbs/06a_memory.experience_replay.ipynb
converting: /home/fastrl_user/fastrl/nbs/06f_memory.tensorboard.ipynb
An error occurred while e